## Recall of Self Attention

For a query row of attention weights $x_1, x_2, ..., x_N$, denote
* the running max $m_i = \max_{j=1}^{i - 1}(m_j, x_i)$ where $m_1 = x_1$,
* the denominator $d = \sum_{j=1}^N e^{x_j - m_N}$,
* the attention score $a_i = C$,
* and the output states $o = \sum_{j=1}^N a_j v_j $

## Flash Attention
Flash Attention trades increased computation intensity (recursive recompute) for reduced memory reads on GPUs.
This is done by computing above components as running values via recursions.

For denominator, obviously $d_N = d$ when we define its recursive form as follows:
$$
\begin{align}
d_i =& \sum_{j=1}^i e^{x_j - m_i} = (\sum_{j=1}^{i-1} e^{x_j - m_i}) + e^{x_i - m_i} \\
=& (\sum_{j=1}^{i-1} e^{x_j - m_i}) (e^{m_i - m_{i-1}} e^{-m_i + m_{i-1}}) + e^{x_i - m_i} \\
=& (\sum_{j=1}^{i-1} e^{x_j - m_{i-1}} e^{-m_i + m_{i-1}}) + e^{x_i - m_i} \\
=& d_{i-1} \cdot e^{-m_i + m_{i-1}} + e^{x_i - m_i}
\end{align}
$$

For the output states, similarly, $o_N = o$ when we define the following recursive form:
$$
\begin{align}
o_i =& \sum_{j=1}^i a_j v_j = \sum_{j=1}^i \frac{ e^{x_i - m_i} }{d_i} v_j \\
=& (\sum_{j=1}^{i-1} \frac{ e^{x_j - m_i} }{d_i} v_j) + \frac{ e^{x_i - m_i} }{d_i} v_i \\
=& (\sum_{j=1}^{i-1} \frac{ e^{x_j - m_i} }{d_i} v_j) (\frac{ d_{i-1}}{d_{i-1}} \cdot \frac{e^{- m_{i-1}} }{e^{-  m_{i-1}} }) + \frac{ e^{x_i - m_i} }{d_i} v_i \qquad \text{(Make Dummy)} \\
=& (\sum_{j=1}^{i-1} \frac{ e^{x_j - m_{i-1}} }{d_{i-1}} v_j) (\frac{ d_{i-1}}{d_i} \cdot \frac{e^{-m_i} }{e^{-  m_{i-1}} }) + \frac{ e^{x_i - m_i} }{d_i} v_i \qquad \text{(Swap in)} \\
=& o_{i-1} (\frac{ d_{i-1}}{d_i} \cdot \frac{e^{-m_i} }{e^{-  m_{i-1}} }) + \frac{ e^{x_i - m_i} }{d_i} v_i \\
=& o_{i-1} (\frac{ d_{i-1}}{d_i} \cdot e^{-m_i + m_{i-1}})+ \frac{ e^{x_i - m_i} }{d_i} v_i
\end{align}
$$

## Reference

[1] https://www.youtube.com/watch?v=gBMO1JZav44